# Generate simulated data to link

In this case study, we imagine running a person linkage on a simulated 2030 Census Unedited File (CUF) --
see the main notebook for more details, including references used throughout this notebook.
This notebook creates simulated input (CUF) and reference files approximating what would be used in such a process,
emulating the methods found in publicly available descriptions of the Census Bureau's primary person linkage system, PVS.

In [1]:
import pseudopeople as psp
import os
import logging
import warnings
import os
import shutil
from pathlib import Path
# Importing pandas for access, regardless of whether we are using it as the compute engine
import pandas
import numpy

In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
%aimport vivarium_research_prl
from vivarium_research_prl import distributed_compute, utils

In [4]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
# DO NOT EDIT if this notebook is not called generate_simulated_data_small_sample.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
# If running with the default parameters, you can overwrite this notebook; otherwise,
# save it to another filename.
# TODO: Rename the notebook to omit 'small_sample' in the filename and omit all outputs
# from the 'canonical version'
data_to_use = 'small_sample'
output_dir = 'output'
compute_engine = 'pandas'
num_jobs = 10
cpus_per_job = 2
memory_per_job = "10GB"

In [6]:
! date

Thu 08 Feb 2024 08:49:39 AM PST


In [7]:
psp.__version__

'0.8.3.dev6+g31db93a'

In [8]:
df_ops, pd = distributed_compute.start_compute_engine(
    compute_engine,
    num_jobs=num_jobs,
    cpus_per_job=cpus_per_job,
    memory_per_job=memory_per_job,
)

In [9]:
def save_file_with_ground_truth(file_name, file, ground_truth):
    # Check that file and ground truth have the same records
    assert (
        # record_id is unique
        len(file) ==
        len(df_ops.drop_duplicates(file[['record_id']]))
    )
    assert (
        len(ground_truth) ==
        len(file[['record_id']].merge(ground_truth[['record_id']], on='record_id', how='inner'))
    )

    file_path = f'{output_dir}/{data_to_use}/{file_name}.parquet'
    utils.remove_path(file_path)
    file.to_parquet(file_path)

    ground_truth_path = f'{output_dir}/{data_to_use}/{file_name}_ground_truth.parquet'
    utils.remove_path(ground_truth_path)
    ground_truth.to_parquet(ground_truth_path)

## Load pseudopeople simulated datasets

### Record ID tracking (data lineage)

We do a little bit of work here to enable tracking the "ground truth" (the simulant IDs from
pseudopeople).
We give each pseudopeople record/row a unique identifier for tracking, and then we immediately
separate the ground truth information (which we would not have if we were using real data)
from the rest of the columns (which we would have).
The ground truth is only used in the specific "ground truth" section of this notebook,
to help avoid accidentally leaking information into the case study.

Since we also combine/aggregate pseudopeople records as part of the process of generating the
simulated reference files, ground truth is a bit more complicated than you might imagine.
For example, the ground truth may tell us that a single row in a reference file is actually
a composite of several individuals, because even the deterministic linkage (by SSN) we use
here is not without error.

We handle this by tracking *all* source records used in the construction of each record in
our reference files.
This is achieved by having a table mapping composite record IDs to the "source record IDs"
(IDs of records that were directly generated by pseudopeople).
When we aggregate records, this is combined accordingly.

In [10]:
def add_unique_id_col(df, col_name='unique_id', value_prefix=''):
    if compute_engine == 'pandas' or compute_engine.startswith('modin'):
        return df.reset_index().rename(columns={'index': col_name}).assign(**{col_name: lambda df: value_prefix + df[col_name].astype(str)})
    elif compute_engine == 'dask':
        # Can use cumsum as in https://stackoverflow.com/a/60852409/ if it needs
        # to be incrementing, but we just need uniqueness

        def add_id_to_partition(df_part, partition_info=None):
            return (
                df_part
                    .assign(**{col_name: range(len(df_part))})
                    .assign(**{col_name: lambda x: (
                            value_prefix +
                            str(partition_info['number'] if partition_info is not None else 0) +
                            '_' +
                            x[col_name].astype(str)
                        ).astype('large_string[pyarrow]')}
                    )
            )

        df = df.map_partitions(add_id_to_partition)

        return df_ops.ensure_large_string_capacity(df)
    else:
        raise ValueError()

def add_unique_record_id(df, dataset_name):
    return add_unique_id_col(df, col_name='record_id', value_prefix=f'{dataset_name}_')

# Initializes a table listing the pairs between record_ids and source record_ids.
# Should only be called on "source records"; that is, records that
# come directly out of pseudopeople.
def record_id_to_single_source_record_pairs(df, source_col='record_id'):
    if source_col == 'record_id':
        # We can't have duplicate column names, so we make a new column
        # literally called 'source_col'
        df = df.assign(source_col=lambda df: df[source_col])
        source_col = 'source_col'

    return df[['record_id', source_col]].rename(columns={source_col: 'source_record_id'})

In [11]:
# Operations that aggregate records, combining the source record pairs
# between all records that are aggregated

def merge_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix, *args, **kwargs):
    assert len(dfs) == len(source_record_pairings)
    for df in dfs:
        assert 'record_id' in df.columns

    on = kwargs.get('on', None)
    if on is not None:
        # If there are nulls in any of the merge columns, they can't match to anything
        dfs = [df.dropna(subset=on, how='any') for df in dfs]

    result = dfs[0]
    source_record_pairs = source_record_pairings[0]
    dfs_and_source_record_pairs_to_combine = list(zip(dfs[1:], source_record_pairings[1:]))
    for index, (df_to_merge, source_record_pairs_to_merge) in enumerate(dfs_and_source_record_pairs_to_combine):
        result = (
            result.merge(df_to_merge, *args, **kwargs)
        )
        if index == len(dfs_and_source_record_pairs_to_combine) - 1:
            # Since this is the last step, these are the record_ids that will actually be returned
            accumulate_step_record_id_prefix = new_record_id_prefix
        else:
            # A dummy intermediate -- this shouldn't be exposed to the user
            accumulate_step_record_id_prefix = f'merge_iter_{index}'

        result = add_unique_record_id(result, accumulate_step_record_id_prefix)
        source_record_pairs = df_ops.concat([
            # The pairs that were already in result
            source_record_pairs
                .rename(columns={'record_id': 'record_id_x'})
                .merge(result[['record_id', 'record_id_x']], on='record_id_x')
                .drop(columns=['record_id_x']),
            # The new ones
            source_record_pairs_to_merge
                .rename(columns={'record_id': 'record_id_y'})
                .merge(result[['record_id', 'record_id_y']], on='record_id_y')
                .drop(columns=['record_id_y']),
        ])
        result = result.drop(columns=['record_id_x', 'record_id_y'])

    return result, source_record_pairs


def dedupe_preserving_source_records(df, source_record_pairs, columns_to_dedupe, new_record_id_prefix):#, source_records_col='source_record_ids'):
    result = df_ops.drop_duplicates(df[columns_to_dedupe])
    result = add_unique_record_id(result, new_record_id_prefix)
    df_to_result_mapping = (
        df[['record_id'] + columns_to_dedupe]
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(result, on=columns_to_dedupe)
            [['record_id', 'record_id_pre_dedupe']]
    )
    result_source_record_pairs = (
        source_record_pairs
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(df_to_result_mapping, on='record_id_pre_dedupe')
            .drop(columns=['record_id_pre_dedupe'])
    )
    return result, result_source_record_pairs


def concat_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix):
    dfs = [df.rename(columns={'record_id': 'record_id_pre_concat'}) for df in dfs]
    result = df_ops.concat(dfs, ignore_index=True)
    result = add_unique_record_id(result, new_record_id_prefix)

    record_id_mapping = (
        result[['record_id', 'record_id_pre_concat']]
    )
    validate_kwarg = {
        'validate': 'm:1',
    } if compute_engine != 'dask' else {} # Dask doesn't support validate

    all_source_record_pairings = df_ops.persist(df_ops.concat(source_record_pairings, ignore_index=False))
    result_source_record_pairs = df_ops.persist(
        all_source_record_pairings
            .rename(columns={'record_id': 'record_id_pre_concat'})
            .merge(record_id_mapping, on='record_id_pre_concat', **validate_kwarg)
            .drop(columns=['record_id_pre_concat'])
    )

    if compute_engine == 'dask':
        # Manual alternative to the 'validate' kwarg
        assert len(result_source_record_pairs) == len(all_source_record_pairings)

    return result.drop(columns=['record_id_pre_concat']), result_source_record_pairs

### 1040 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files -- see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [12]:
tax_years = list(range(2025, 2030))
tax_years

[2025, 2026, 2027, 2028, 2029]

In [13]:
%%time

# Combine 1040 for all years.
simulated_taxes_1040 = df_ops.concat([
    df_ops.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_1040_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)
simulated_taxes_1040 = df_ops.persist(add_unique_record_id(simulated_taxes_1040, 'simulated_1040'))
simulated_taxes_1040_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_taxes_1040))

CPU times: user 238 ms, sys: 103 ms, total: 341 ms
Wall time: 254 ms


In [14]:
simulated_taxes_1040_ground_truth = simulated_taxes_1040[['record_id', 'simulant_id']]
simulated_taxes_1040 = simulated_taxes_1040.drop(columns=['simulant_id'])
simulated_taxes_1040, simulated_taxes_1040_ground_truth = df_ops.persist(simulated_taxes_1040, simulated_taxes_1040_ground_truth)
simulated_taxes_1040

,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,simulated_1040_0,794-23-1522,Heather,J,Campusano Robledo,None,None,None,14011,Anytown,WA,00000
1,simulated_1040_1,004-53-9627,Bailey,J,Nicholas,1934,cypress ave,None,None,Anytown,WA,00000
2,simulated_1040_2,273-52-7750,Heather,S,Blackwell,57,shadowood dr,None,None,Anytown,WA,00000
3,simulated_1040_3,384-45-8523,Candy,S,Luedeman,10881,maple av,None,None,Anytown,WA,00000
4,simulated_1040_4,858-84-4517,Patrick,J,Morfin,103,71st st w,None,None,Anytown,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...
39114,simulated_1040_39114,595-04-6651,Akari,L,Parkinson,2921,curtis la,None,None,Anytown,WA,00000
39115,simulated_1040_39115,523-63-9324,Jada,T,Franco Resendez,1920,white ibis ct,None,None,Anytown,WA,00000
39116,simulated_1040_39116,187-01-0690,Bianca,L,Sargent,12613,teppert,None,None,Anytown,WA,00000
39117,simulated_1040_39117,808-04-3639,Ashton,N,Cardenas,116,wakema rd,None,None,Anytown,WA,00000


### W2/1099 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files --
see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [15]:
%%time

# Combine W2/1099 for all years.
simulated_w2_1099 = df_ops.concat([
    df_ops.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_w2_and_1099_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)
simulated_w2_1099 = df_ops.persist(add_unique_record_id(simulated_w2_1099, 'simulated_w2_1099'))
simulated_w2_1099_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_w2_1099))

CPU times: user 258 ms, sys: 83.5 ms, total: 342 ms
Wall time: 241 ms


In [16]:
simulated_w2_1099_ground_truth = simulated_w2_1099[['record_id', 'simulant_id']]
simulated_w2_1099 = simulated_w2_1099.drop(columns=['simulant_id'])
simulated_w2_1099, simulated_w2_1099_ground_truth = df_ops.persist(simulated_w2_1099, simulated_w2_1099_ground_truth)
simulated_w2_1099

,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,simulated_w2_1099_0,854-13-6295,Erin,R,No,1501,interlake ave n,None,None,Anytown,WA,00000
1,simulated_w2_1099_1,854-13-6295,Erin,R,Ztark Llzano,1501,interlake ave n,None,None,Anytown,WA,00000
2,simulated_w2_1099_2,794-23-1522,Heather,J,Campusano Robledo,None,None,None,14011,Anytown,WA,00000
3,simulated_w2_1099_3,093-85-7512,Melisa,D,Velazquez,627,broomfield ln,None,None,Anytown,WA,00000
4,simulated_w2_1099_4,093-85-3512,Melisa,D,Velazquez,927,broomfield ln,None,None,Anytown,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...
53637,simulated_w2_1099_53637,119-02-0252,Paul,None,Hurt,3515,javello rd,None,None,Anytown,WA,00000
53638,simulated_w2_1099_53638,079-75-0734,Lori,T,Washington,9358,e 58th st,None,None,Anytown,WA,00000
53639,simulated_w2_1099_53639,479-75-9734,Lori,None,Washington,9358,e 58th st,None,None,Anytown,WA,00000
53640,simulated_w2_1099_53640,287-19-2431,Patricia,C,Ehle,None,rdg park dr,None,None,Anytown,WA,00000


In [17]:
simulated_taxes, simulated_taxes_source_record_pairs = concat_preserving_source_records(
    [simulated_taxes_1040, simulated_w2_1099],
    [simulated_taxes_1040_source_record_pairs, simulated_w2_1099_source_record_pairs],
    new_record_id_prefix='simulated_taxes',
)

In [18]:
simulated_taxes_1040_with_itins = simulated_taxes_1040[simulated_taxes_1040.ssn.notnull() & simulated_taxes_1040.ssn.str.startswith('9')]
simulated_taxes_1040_with_itins_source_record_pairs = simulated_taxes_1040_source_record_pairs.merge(simulated_taxes_1040_with_itins[['record_id']], on='record_id', how='inner')
simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs = df_ops.persist(simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs)

%xdel simulated_taxes_1040
%xdel simulated_taxes_1040_source_record_pairs

In [19]:
%xdel simulated_w2_1099
%xdel simulated_w2_1099_source_record_pairs

In [20]:
# "... many of the [IRS] records contain only the first four letters of the last name."
# (Brown et al. 2023, p.30, footnote 19)
# This should be updated in pseudopeople but for now we do it here.
# Note that this truncation only matters for ITIN PIKing since for SSNs that are present in SSA we use name from SSA.
PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION = 0.4 # is this a good guess at "many" in the quote above?
if compute_engine == 'dask':
    # import dask.array as np
    # rng = np.random.default_rng(seed=1234)
    # random_df = pd.from_dask_array(rng.random(len(simulated_taxes), chunks=int(np.ceil(len(simulated_taxes) / simulated_taxes.npartitions))), columns=['random'])
    # random_df.index = simulated_taxes.index
    # to_truncate = simulated_taxes.assign(random=random_df.random).random < PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION
    to_truncate, not_to_truncate = simulated_taxes.random_split([PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION, 1 - PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION], random_state=1234)
    to_truncate = to_truncate.assign(last_name=to_truncate.last_name.str[:4])
    simulated_taxes = df_ops.concat([to_truncate, not_to_truncate], ignore_index=True)
else:
    import numpy as np
    to_truncate = simulated_taxes.sample(frac=PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION, random_state=1234).index

    simulated_taxes.loc[to_truncate, 'last_name'] = simulated_taxes.loc[to_truncate, 'last_name'].str[:4]

In [21]:
simulated_taxes, simulated_taxes_source_record_pairs = df_ops.persist(simulated_taxes, simulated_taxes_source_record_pairs)

### 2030 Census Unedited File (CUF)

For now, we gloss over the data schema for addresses.
We don't know how addresses would be formatted in the CUF (and it's hard to guess, because
address is not part of the Census form), but it likely would have some of these fields
(street number, street name, etc) combined.

While PVS input files do not in general have names split into first, middle, and last,
I am guessing the CUF **would** have first name, middle initial, last name (which is how pseudopeople
generates it), because that [matches the Census questionnaire](https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/questionnaires-and-instructions/questionnaires/2020-informational-questionnaire-english_DI-Q1.pdf).

In [22]:
%%time

simulated_census_2030 = df_ops.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_census_2030.parquet')
simulated_census_2030 = df_ops.persist(add_unique_record_id(simulated_census_2030, 'simulated_census_2030'))

CPU times: user 84.9 ms, sys: 11.5 ms, total: 96.4 ms
Wall time: 60.7 ms


In [23]:
simulated_census_2030_ground_truth = simulated_census_2030[['record_id', 'simulant_id']]
simulated_census_2030 = simulated_census_2030.drop(columns=['simulant_id'])
simulated_census_2030, simulated_census_2030_ground_truth = df_ops.persist(simulated_census_2030, simulated_census_2030_ground_truth)
simulated_census_2030

,record_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year
0,simulated_census_2030_0,0_8033,Gerald,R,Allen,86,11/03/1943,1130,mallory ln,None,Anytown,WA,00000,Household,Reference person,Male,Black,2030
1,simulated_census_2030_1,0_1066,April,S,Hayden,33,10/23/1996,32597,delacorte dr,None,Anytown,WA,00000,Household,Other nonrelative,Female,Black,2030
2,simulated_census_2030_2,0_1066,Loretta,T,Lowe,71,06/01/1958,32597,delacorte dr,None,Anytown,WA,00000,Household,Reference person,Female,White,2030
3,simulated_census_2030_3,0_2514,Sandra,A,Sorrentino,75,03/18/1954,4458,wibdsor pl,None,Anytown,WA,00000,Household,Reference person,Female,Multiracial or Other,2030
4,simulated_census_2030_4,0_5627,Bobby,S,Baker,44,05/20/1985,None,winding trail rd,None,Anytown,WA,00000,Household,Other nonrelative,Male,White,2030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11024,simulated_census_2030_11024,0_10778,Jeremy,T,Boyd,46,07/01/1983,211,quiet wsy,None,Anytown,WA,00000,Household,Reference person,Male,Black,2030
11025,simulated_census_2030_11025,0_11001,Wendy,M,Gross,54,12/05/1975,2801,blje rdv dr n,None,Anytown,WA,00000,Household,Reference person,Female,White,2030
11026,simulated_census_2030_11026,0_5308,Ember,H,Samuels,10,10/26/2019,24113,lauder,None,Anytown,WA,00000,Household,Reference person,Female,Black,2030
11027,simulated_census_2030_11027,0_10693,Athena,V,Deshpande,27,07/05/2002,1534,bentley dr,None,Anytown,WA,00000,Household,Reference person,Female,Asian,2030


In [24]:
save_file_with_ground_truth('simulated_census_2030', simulated_census_2030, simulated_census_2030_ground_truth)

In [25]:
%xdel simulated_census_2030

### Simulated SSA Numident

In [26]:
%%time

simulated_ssa_numident = df_ops.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_ssa_numident.parquet')

CPU times: user 69.9 ms, sys: 23.6 ms, total: 93.5 ms
Wall time: 58.7 ms


In [27]:
if data_to_use == 'ri':
    # Remove simulants who were never observed in Rhode Island -- this is unrealistic to real life!
    # We only do this to create a medium-sized set of reference files for computational testing.
    simulated_ssa_numident = (
        simulated_ssa_numident
            .merge(
                df_ops.drop_duplicates(simulated_taxes_1040_ground_truth[['simulant_id']]).assign(observed_in_taxes_1040=1),
                on='simulant_id',
                how='left',
            )
            .merge(
                df_ops.drop_duplicates(simulated_w2_1099_ground_truth[['simulant_id']]).assign(observed_in_taxes_w2_1099=1),
                on='simulant_id',
                how='left',
            )
            .merge(
                df_ops.drop_duplicates(simulated_census_2030_ground_truth[['simulant_id']]).assign(observed_in_census_2030=1),
                on='simulant_id',
                how='left',
            )
            .assign(observed_ever=lambda df: (df.observed_in_taxes_1040 == 1) | (df.observed_in_taxes_w2_1099 == 1) | (df.observed_in_census_2030 == 1))
            .pipe(lambda df: df[df.observed_ever])
            .drop(columns=["observed_in_taxes_1040", "observed_in_taxes_w2_1099", "observed_in_census_2030", "observed_ever"])
    )

In [28]:
%xdel simulated_census_2030_ground_truth

In [29]:
simulated_ssa_numident = df_ops.persist(add_unique_record_id(simulated_ssa_numident, 'simulated_ssa_numident'))
simulated_ssa_numident_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_ssa_numident))

In [30]:
simulated_ssa_numident_ground_truth = simulated_ssa_numident[['record_id', 'simulant_id']]
simulated_ssa_numident = simulated_ssa_numident.drop(columns=['simulant_id'])
simulated_ssa_numident, simulated_ssa_numident_ground_truth = df_ops.persist(simulated_ssa_numident, simulated_ssa_numident_ground_truth)
simulated_ssa_numident

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date
0,simulated_ssa_numident_0,786-77-6454,Evelyn,Nancy,Hines,19191204,Female,creation,19191204
1,simulated_ssa_numident_1,688-88-6377,George,Robert,Dickens,19210616,Male,creation,19210616
2,simulated_ssa_numident_2,651-33-9561,Beatrice,Jennie,Fackler,19220113,Female,creation,19220113
3,simulated_ssa_numident_3,665-25-7858,Eura,Nadine,Crusen,19220305,Female,creation,19220305
4,simulated_ssa_numident_4,875-10-2359,Roberta,Ruth,Muilenburg,19220306,Female,creation,19220306
...,...,...,...,...,...,...,...,...,...
19866,simulated_ssa_numident_19866,203-10-6855,Birdie,Lizzy,Mercer,None,Female,creation,10291223
19867,simulated_ssa_numident_19867,505-88-8814,Levi,Holden,Smal,None,Male,creation,20291224
19868,simulated_ssa_numident_19868,417-18-3811,Alan,Messiah,Bazan,20291227,Male,creation,23293227
19869,simulated_ssa_numident_19869,450-74-3175,Emmi,Everlynn,H,20291228,Female,creation,20191228


In [31]:
source_record_ground_truth = df_ops.persist(df_ops.concat([
    simulated_ssa_numident_ground_truth,
    simulated_taxes_1040_ground_truth,
    simulated_w2_1099_ground_truth,
]).rename(columns={'record_id': 'source_record_id'}))
source_record_ground_truth

,source_record_id,simulant_id
0,simulated_ssa_numident_0,0_19979
1,simulated_ssa_numident_1,0_6846
2,simulated_ssa_numident_2,0_19983
3,simulated_ssa_numident_3,0_262
4,simulated_ssa_numident_4,0_12473
...,...,...
53637,simulated_w2_1099_53637,0_8910
53638,simulated_w2_1099_53638,0_2378
53639,simulated_w2_1099_53639,0_2378
53640,simulated_w2_1099_53640,0_22909


In [32]:
%xdel simulated_taxes_1040_ground_truth
%xdel simulated_w2_1099_ground_truth

## Create reference files

> The Census Numident – all Social Security Administration (SSA) Numident SSN records are
  edited (collapsed) to produce a Census Numident file that contains “one best-data record” for
  each SSN. All variants of name information for each SSN are retained in the Alternate Name
  Numident file, while all variants of date of birth data are retained in the Alternate DOB
  Numident. The SSN-PIK crosswalk file is used to attach a corresponding unique PIK value for
  each SSN value in the Census Numident file.

### Census Numident

Luque and Wagner, p. 4:
  
> The SSA Numident file contains all transactions ever recorded against any single SSN - with each entry
representing an addition or change (such as name changes) to the SSN record. This file is edited to
create the **Census Numident**, which contains one record for each SSN. Each SSN record in the Census
Numident contains name, DOB, sex, race, place of birth, parents’ name, citizenship status and date of death information.

and in footnote 5:

> Name edits, DOB reconciliation, and race identifiers are some of the edits conducted to produce this Numident
file. **The resulting Numident file contains the most recent name and DOB data.**

We are missing quite a few columns, since they are missing in pseudopeople's SSA Numident: race, place of birth, parents' name,
citizenship status.
However, I'm pretty sure none of these are used in linking.

In [33]:
def fill_dates(df, fill_with):
    return (
        # Replace invalid dates with nans
        pd.to_datetime(df.event_date, format='%Y%m%d', errors='coerce')
            .fillna(pandas.to_datetime('2100-01-01' if fill_with == 'latest' else '1900-01-01'))
    )

def best_data_from_columns(df, columns, best_is_latest=True):
    # We don't want to throw out events with a missing/invalid date, so we'll fill them with the value *least* likely to be chosen
    # (earlier than all values if taking the latest, later than all values if taking the earliest).
    fill_with = 'earliest' if best_is_latest else 'latest'

    result = (
        df
            # Without mutating the existing date column, get one that is actually
            # a date type and can be used for sorting.
            # Note: we actually convert this to an integer for sorting purposes, because Modin was having trouble
            # sorting by it as an actual datetime
            .assign(event_date_for_sort=lambda df: fill_dates(df, fill_with=fill_with).astype(numpy.int64) // 10 ** 9)
            .dropna(subset=columns, how='all')
            .pipe(lambda df: df_ops.drop_duplicates(df, subset='ssn', sort_col='event_date_for_sort', keep=('last' if best_is_latest else 'first')))
            [['record_id', 'ssn'] + columns]
    )

    return result, record_id_to_single_source_record_pairs(result)

best_name, best_name_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['first_name', 'middle_name', 'last_name'],
)

best_date_of_birth, best_date_of_birth_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['date_of_birth'],
)

best_date_of_death, best_date_of_death_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident[simulated_ssa_numident.event_type == 'death'],
    columns=['event_date'],
)
best_date_of_death = best_date_of_death.rename(columns={'event_date': 'date_of_death'})

simulated_census_numident, simulated_census_numident_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [best_name, best_date_of_birth, best_date_of_death],
    [best_name_source_record_pairs, best_date_of_birth_source_record_pairs, best_date_of_death_source_record_pairs],
    new_record_id_prefix='simulated_census_numident',
    on=['ssn'],
    how='left',
))
simulated_census_numident

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,date_of_death
0,simulated_census_numident_0,192-40-6580,Evan,Jonathan,Herz,19871027,NaN
1,simulated_census_numident_1,336-80-8181,Debra,Olivia,Thomas,19911124,NaN
2,simulated_census_numident_2,226-42-4899,Brian,Uake,Cay,19920504,NaN
3,simulated_census_numident_3,016-17-1944,Ariel,Traci,Stachon,19920809,NaN
4,simulated_census_numident_4,297-33-2201,Paul,Christopher,Blake,19924508,NaN
...,...,...,...,...,...,...,...
18464,simulated_census_numident_18464,765-50-4823,Marsha,Elizabeth,Perez,19540922,NaN
18465,simulated_census_numident_18465,626-01-0203,Aria,Roxana,Strain,20230202,NaN
18466,simulated_census_numident_18466,333-36-9813,Charles,Michael,Nguyen,19990611,NaN
18467,simulated_census_numident_18467,093-26-7943,Hansel,Xander,Valenzuela,20240328,NaN


### Alternate Name Numident

Wagner and Layne, p. 9:

>  All variants of name information for each SSN are retained in the Alternate Name
Numident file...

In [34]:
simulated_alternate_name_numident, simulated_alternate_name_numident_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_alternate_name_numident',
))
simulated_alternate_name_numident

,record_id,ssn,first_name,middle_name,last_name
0,simulated_alternate_name_numident_11,685-77-0916,Betty,Audrey,Keel
1,simulated_alternate_name_numident_14,765-44-4521,Ethep,Nancy,Collier
2,simulated_alternate_name_numident_20,726-57-2168,Josephine,Margaret,Babbie
3,simulated_alternate_name_numident_21,365-44-3027,Betty,Mary,None
4,simulated_alternate_name_numident_22,512-15-1742,Martha,Beulah,Burandt
...,...,...,...,...,...
18918,simulated_alternate_name_numident_19866,203-10-6855,Birdie,Lizzy,Mercer
18919,simulated_alternate_name_numident_19867,505-88-8814,Levi,Holden,Smal
18920,simulated_alternate_name_numident_19868,417-18-3811,Alan,Messiah,Bazan
18921,simulated_alternate_name_numident_19869,450-74-3175,Emmi,Everlynn,H


In [35]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_alternate_name_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    18469.000000
mean         1.024582
std          0.154851
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [36]:
# Show alternate names for the same simulated SSN
simulated_alternate_name_numident.merge(
    df_ops.groupby_agg_small_groups(simulated_alternate_name_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,record_id,ssn,first_name,middle_name,last_name
539,simulated_alternate_name_numident_16381,796-24-3260,Donna,Renee,Altschuler
538,simulated_alternate_name_numident_1767,796-24-3260,Donna,None,Altschuler
905,simulated_alternate_name_numident_17540,796-63-9594,Steven,Michael,Ridley
904,simulated_alternate_name_numident_16760,796-63-9594,Steven,Michael,Ridleh
540,simulated_alternate_name_numident_1782,798-12-1969,Sandra,Linda,Turner
...,...,...,...,...,...
901,simulated_alternate_name_numident_16331,898-71-7849,Fay,Jean,Woldridge
682,simulated_alternate_name_numident_2971,899-07-0109,Jeffrey,Glenn,Haqkins
683,simulated_alternate_name_numident_17752,899-07-0109,Jeffrey,Glenn,Hawkins
591,simulated_alternate_name_numident_18365,899-24-9400,Willie,David,Laduke


### Alternate DOB Numident

Wagner and Layne, p. 9:

> ... while all variants of date of birth data are retained in the Alternate DOB
Numident.

In [37]:
simulated_alternate_dob_numident, simulated_alternate_dob_numident_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'date_of_birth'],
    new_record_id_prefix='simulated_alternate_dob_numident',
))
simulated_alternate_dob_numident

,record_id,ssn,date_of_birth
0,simulated_alternate_dob_numident_2,651-33-9561,19220113
1,simulated_alternate_dob_numident_14,765-44-4521,None
2,simulated_alternate_dob_numident_26,898-96-4862,19240302
3,simulated_alternate_dob_numident_30,075-01-3445,19240610
4,simulated_alternate_dob_numident_34,027-46-6956,19240918
...,...,...,...
18724,simulated_alternate_dob_numident_19866,203-10-6855,None
18725,simulated_alternate_dob_numident_19867,505-88-8814,None
18726,simulated_alternate_dob_numident_19868,417-18-3811,20291227
18727,simulated_alternate_dob_numident_19869,450-74-3175,20291228


In [38]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_alternate_dob_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    18469.000000
mean         1.014078
std          0.117814
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [39]:
simulated_alternate_dob_numident.merge(
    df_ops.groupby_agg_small_groups(simulated_alternate_dob_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,record_id,ssn,date_of_birth
49,simulated_alternate_dob_numident_17745,751-96-8435,19320905
48,simulated_alternate_dob_numident_184,751-96-8435,19320509
504,simulated_alternate_dob_numident_11691,752-86-2170,20013511
505,simulated_alternate_dob_numident_16765,752-86-2170,20010511
195,simulated_alternate_dob_numident_16483,757-47-3233,20444239
...,...,...,...
511,simulated_alternate_dob_numident_16331,898-71-7849,19320809
5,simulated_alternate_dob_numident_17910,898-96-4862,19240002
4,simulated_alternate_dob_numident_26,898-96-4862,19240302
375,simulated_alternate_dob_numident_17752,899-07-0109,19580911


### Name/DOB Reference File

Wagner and Layne, p. 9:

> The Name and DOB Reference files are reformatted versions of the Census Numident
and includes **all possible combinations of alternate names and dates of birth, as well as
ITIN data**. All of the reference files contain SSN/ITIN and the corresponding PIK. When
an input record is linked to a reference file, the corresponding PIK is assigned. Table 1
presents the number of observations in each of the reference files.

A slightly confusing point: sometimes the Name and DOB reference files are described
as one and the same thing, and sometimes as separate.
I believe this is because **they differ only in how they are "cut" for the PVS process:**
the name reference file is cut by first and last initial,
while the DOB reference file is cut by month and day of birth.

This is described in Wagner and Layne, p.15:

> The [DOBSearch] module matches against a re-split
version of the Numident Name Reference file, splitting the data based on month and day
of birth.

Since we handle the logic of "cutting" in the linkage process itself, we generate
a single reference file here.

Note that unlike for addresses, and unlike for the pre-processing of *input* files
(as opposed to reference files), there is no explicit nickname processing/correction here.
I am fairly sure that is accurate to the PVS, which I believe assumes that nicknames
would not be present in SSA/tax records (or at least, that the real name would appear
at least once in these records).

In [40]:
simulated_name_dob_numident_records, simulated_name_dob_numident_records_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [simulated_alternate_name_numident, simulated_alternate_dob_numident],
    [simulated_alternate_name_numident_source_record_pairs, simulated_alternate_dob_numident_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='name_dob_numident_records',
))
simulated_name_dob_numident_records

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
0,name_dob_numident_records_0,685-77-0916,Betty,Audrey,Keel,19221205
1,name_dob_numident_records_1,765-44-4521,Ethep,Nancy,Collier,None
2,name_dob_numident_records_2,765-44-4521,Ethep,Nancy,Collier,19230213
3,name_dob_numident_records_3,726-57-2168,Josephine,Margaret,Babbie,19230728
4,name_dob_numident_records_4,365-44-3027,Betty,Mary,None,19230809
...,...,...,...,...,...,...
19262,name_dob_numident_records_19262,203-10-6855,Birdie,Lizzy,Mercer,None
19263,name_dob_numident_records_19263,505-88-8814,Levi,Holden,Smal,None
19264,name_dob_numident_records_19264,417-18-3811,Alan,Messiah,Bazan,20291227
19265,name_dob_numident_records_19265,450-74-3175,Emmi,Everlynn,H,20291228


In [41]:
# Show alternate name/DOB for the same simulated SSN
simulated_name_dob_numident_records.merge(
    df_ops.groupby_agg_small_groups(simulated_name_dob_numident_records[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner',
).sort_values('ssn').tail(100)

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
1154,name_dob_numident_records_16151,838-68-3979,Linda,Maxine,Whitehead,None
1155,name_dob_numident_records_16152,838-68-3979,Linda,Maxine,Whitehead,19401105
1097,name_dob_numident_records_15292,838-91-4443,Bruce,Leroy,Vossen,19420114
1096,name_dob_numident_records_15291,838-91-4443,Bruce,Leroy,Vossen,19425114
565,name_dob_numident_records_1088,839-37-3165,Linda,Caryn,None,19490818
...,...,...,...,...,...,...
803,name_dob_numident_records_2367,899-07-0109,Jeffrey,Glenn,Haqkins,19580911
804,name_dob_numident_records_17001,899-07-0109,Jeffrey,Glenn,Hawkins,19650509
805,name_dob_numident_records_17002,899-07-0109,Jeffrey,Glenn,Hawkins,19580911
697,name_dob_numident_records_17643,899-24-9400,Willie,David,Laduke,19530914


#### Incorporating people with ITINs

Individual Taxpayer Identification Numbers (ITINs) can be issued to people who are required to file
federal taxes but are not eligible for a Social Security Number.
The most common reason for this is being an undocumented immigrant and therefore not being authorized
to work in the United States.

People without SSNs used to be impossible to assign PIKs to.
In 2011 the NORC report stated (p. 38, footnote 19):

> NORC understands that the Census Bureau has undertaken an effort to enhance the PVS reference files with IRS
files that include Individual Taxpayer Identification Numbers (ITIN). For those people who are required to file a tax
return but do not have, and may not want an SSN—such as a non-U.S. citizen—the IRS issues the taxpayer an ITIN.
This enhancement to the PVS reference file may help to match more non-U.S citizens.

By 2014 (Wagner and Layne, p. 5):

> One of the key enhancements [made in recent years] increased the coverage of the reference files by
including records for persons with Individual Taxpayer Identification Numbers assigned
by the Internal Revenue Service (ITINs) to [along with?] the SSN-based Numident data. 

I have not found a specific description of how ITIN records are constructed in any of the
publicly-available sources.
This may be because it is straightforward, or because the tax data schema is confidential.
I assume that only IRS data is used, since no other data source that I am aware of would
report ITIN.

It is stated that the ITIN records are created directly from tax filings and not
from ITIN applications (Brown et al. p. 29, footnote 16), which is convenient
because the tax filing data is what we can simulate with
pseudopeople:

> The NUMIDENT provides the PII on the SSN-holder from the issuing agency (SSA), and that PII is used in SSN
verification. **For ITINs, the Census Bureau does not have access to the ITIN applications** to the issuing agency (IRS),
so name and DOB verification of ITINs is less reliable.

"Less reliable" is a bit confusing here, because as stated above when generating
the simulated tax data, IRS data should not contain date of birth at all.
Here, we have stayed true to this by omitting it entirely.

We assume that only 1040 filings would be used for this purpose; we wouldn't expect ITINs to
show up on employer-filed W-2/1099 forms.

In [42]:
# Analogous to the process of getting alternate names and dates of birth
# from SSA, we retain all versions of the name from taxes.
simulated_name_for_itins, simulated_name_for_itins_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_taxes_1040_with_itins.rename(columns={'middle_initial': 'middle_name'}),
    simulated_taxes_1040_with_itins_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_name_for_itins',
))
simulated_name_for_itins

,record_id,ssn,first_name,middle_name,last_name
0,simulated_name_for_itins_93,905-55-4311,Pate,R,Brown-Villarreal
1,simulated_name_for_itins_112,970-57-8403,Arthur,M,None
2,simulated_name_for_itins_133,978-95-3331,Stephen,P,Occupant
3,simulated_name_for_itins_378,917-62-7164,Susie,B,Thomas
4,simulated_name_for_itins_506,921-53-9692,Toni,L,Andreadiz
...,...,...,...,...,...
603,simulated_name_for_itins_39032,977-84-6791,Liam,L,Wheatley
604,simulated_name_for_itins_39075,997-95-0405,Joseph,D,Dellinger
605,simulated_name_for_itins_39080,939-53-1702,Josiah,L,Mcmiller
606,simulated_name_for_itins_39086,993-56-2286,Larisa,A,Beilfuss


In [43]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_name_for_itins, by='ssn', agg_func=lambda x: x.size()).describe())

count    510.000000
mean       1.192157
std        0.432402
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        4.000000
dtype: float64

In [44]:
# How many records in the simulated name/DOB numident are in the ITIN range?
# With normal pseudopeople settings, this shouldn't happen, because SSN in SSA is noiseless.
# If this number is >0, we may be adding ITIN records as more alternates to existing records.
df_ops.compute(simulated_name_dob_numident_records.ssn.str.startswith('9').sum())

0

In [45]:
simulated_name_dob_reference_file, simulated_name_dob_reference_file_source_record_pairs = df_ops.persist(concat_preserving_source_records(
    [simulated_name_dob_numident_records, simulated_name_for_itins],
    [simulated_name_dob_numident_records_source_record_pairs, simulated_name_for_itins_source_record_pairs],
    new_record_id_prefix='simulated_name_dob_reference_file',
))
simulated_name_dob_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
0,simulated_name_dob_reference_file_0,685-77-0916,Betty,Audrey,Keel,19221205
1,simulated_name_dob_reference_file_1,765-44-4521,Ethep,Nancy,Collier,None
2,simulated_name_dob_reference_file_2,765-44-4521,Ethep,Nancy,Collier,19230213
3,simulated_name_dob_reference_file_3,726-57-2168,Josephine,Margaret,Babbie,19230728
4,simulated_name_dob_reference_file_4,365-44-3027,Betty,Mary,None,19230809
...,...,...,...,...,...,...
19870,simulated_name_dob_reference_file_19870,977-84-6791,Liam,L,Wheatley,NaN
19871,simulated_name_dob_reference_file_19871,997-95-0405,Joseph,D,Dellinger,NaN
19872,simulated_name_dob_reference_file_19872,939-53-1702,Josiah,L,Mcmiller,NaN
19873,simulated_name_dob_reference_file_19873,993-56-2286,Larisa,A,Beilfuss,NaN


### GeoBase Reference File

Wagner and Layne, p. 9:

> PVS creates three other sets of reference
files containing Numident data: the **GeoBase Reference File**, the Name Reference File,
and the DOB Reference file.
The GeoBase Reference File appends addresses from administrative records attached
to Numident data, including all possible combinations of alternate names and dates of
birth for SSN. Addresses from administrative records are edited and processed through
commercial software product to clean and standardize address data. ITIN data is also
incorporated into the Geobase.

Luque and Wagner, p. 5:

> Reference files contain data from the Numident file enhanced with address
data obtained from federal AR [administrative records] files.<sup>8</sup>
The reference files, thus, contain all variants of a person’s name, DOB,
and sex, as well as current and recent addresses. These reference files are
referred to as the (PVS) Geobase reference file since addresses (a geographic component)
are appended to each person record.<sup>9</sup> It is important to note that there are
multiple Geobase reference files that are created depending on the vintage of the
incoming file to be processed through PVS.

> <sup>8</sup> Namely, data from the IRS, Department of Housing and Urban Development,
several files from the Department of Health and Human Services, and Selective Service.

> <sup>9</sup> In particular, the address data is cleaned and standardized and used
to construct a variable called GEOKEY. The GEOKEY variable is constructed as a subset
of the full address, and then is appended to the Numident data to create the
PVS Geobase Reference file.

We only have IRS data to use for addresses, and specifically only W-2/1099 data,
which is a limitation of this case study.
I can't find a concrete definition of "recent" -- as noted above, we use 5 years
of IRS data.
This may be longer than the true window, but this may end up making up for
the lack of non-IRS data sources, and get us closer to a realistic number of
alternate addresses.

Also, our address data comes out of pseudopeople already parsed into address parts
like street name, etc.
For more realism, pseudopeople should output a single string that we have to (imperfectly) parse apart.

I haven't been able to find out more about what kind of "subset" the geokey is.
It is unclear to me why geokey is "interesting" since it is just derived from the
address parts.

In [46]:
address_cols = [c for c in simulated_taxes.columns if 'mailing_address' in c]

def standardize_address_part(column):
    return (
        column
            # Remove leading or trailing whitespace
            .str.strip()
            # Turn any strings of consecutive whitespace into a single space
            .str.replace('\s+', ' ', regex=True)
            # Normalize case
            .str.upper()
            # Normalize the word street as described in the example quoted above
            # In reality, there would be many rules like this
            .str.replace('\b(STREET|STR)\b', 'ST', regex=True)
            # Make sure missingness is represented consistently
            .replace('', numpy.nan)
    )

simulated_tax_addresses = (
    simulated_taxes
        # Can only link these to the other files if they have an SSN
        .dropna(subset=['ssn'])
        [['record_id', 'ssn'] + address_cols]
        .assign(**{c: lambda df, c=c: standardize_address_part(df[c]) for c in address_cols})
        .pipe(df_ops.ensure_large_string_capacity)
)

simulated_addresses_by_ssn, simulated_addresses_by_ssn_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_tax_addresses,
    simulated_taxes_source_record_pairs,
    columns_to_dedupe=['ssn'] + address_cols,
    new_record_id_prefix='addresses_by_ssn',
))
simulated_addresses_by_ssn

,record_id,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,addresses_by_ssn_10,172-15-1292,9808,111 AGEHUE,None,None,ANYTOWN,WA,00000
1,addresses_by_ssn_16,19z-24-265G,None,None,None,6846,ANYTOWN,WA,00000
2,addresses_by_ssn_19,606-92-2881,2327,N COLLEGE AFE,None,None,ANYTOWN,WA,00000
3,addresses_by_ssn_21,678-32-5290,None,E CALHOUN ST,None,None,ANYTOWN,WA,00000
4,addresses_by_ssn_25,327-51-5264,9706,N AVONDALE DR,None,None,ANYTOWN,WA,00000
...,...,...,...,...,...,...,...,...,...
31819,addresses_by_ssn_92756,119-02-0252,3515,JAVELLO RD,None,None,ANYTOWN,WA,00000
31820,addresses_by_ssn_92757,079-75-0734,9358,E 58TH ST,None,None,ANYTOWN,WA,00000
31821,addresses_by_ssn_92758,479-75-9734,9358,E 58TH ST,None,None,ANYTOWN,WA,00000
31822,addresses_by_ssn_92759,287-19-2431,None,RDG PARK DR,None,None,ANYTOWN,WA,00000


In [47]:
num_addresses = df_ops.groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).rename('size').reset_index().sort_values('size')
num_addresses

,ssn,size
47,002-98-8305,1
19298,z07-4S-3s24,1
34,002-40-7232,1
35,002-43-9287,1
36,002-51-2470,1
...,...,...
693,034-83-5590,7
3786,186-14-1589,7
3760,184-98-3128,7
10983,531-78-0175,8


In [48]:
# Show some SSNs with a lot of address variation
df_ops.compute(pd.merge(num_addresses.tail(10), simulated_addresses_by_ssn, on='ssn', how='inner').sort_values('ssn'))

,ssn,size,record_id,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
35,034-83-5590,7,addresses_by_ssn_18016,11818,ARBOR MEADOW DRIVE,None,None,ANYTOWN,CO,00000
41,034-83-5590,7,addresses_by_ssn_86294,13322,NW 56 AVE,None,None,ANYTOWN,WA,00000
40,034-83-5590,7,addresses_by_ssn_80642,11818,ARBOR MEADOW DRIVE,None,None,ANYTOWN,WA,00000
39,034-83-5590,7,addresses_by_ssn_69913,11818,ARBOR MEADOW DRIVE,None,None,None,WA,00000
38,034-83-5590,7,addresses_by_ssn_59105,11818,ARBOR MEADOW DRIVE,None,None,ANYTOWN,IN,00000
...,...,...,...,...,...,...,...,...,...,...
29,734-84-5703,7,addresses_by_ssn_23269,18,FRUITRIDGE RD,None,None,None,WA,00000
30,734-84-5703,7,addresses_by_ssn_40654,18,FRU'LTRIDGE RD,None,None,ANYTOWN,WA,00000
31,734-84-5703,7,addresses_by_ssn_51313,18,FRUITRIDGE RD,None,None,ANYTOWN,WA,00000
32,734-84-5703,7,addresses_by_ssn_61846,18,FRUIDRIDGE RD,None,None,ANYTOWN,WA,00000


In [49]:
# Rough estimate of how many rows we should have in our reference file, once we do this Cartesian product
df_ops.compute(
    len(simulated_name_dob_reference_file) *
    df_ops.groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).mean()
)

32756.849137707803

In [50]:
simulated_geobase_reference_file, simulated_geobase_reference_file_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [simulated_name_dob_reference_file, simulated_addresses_by_ssn],
    [simulated_name_dob_reference_file_source_record_pairs, simulated_addresses_by_ssn_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='simulated_geobase_reference_file',
))
simulated_geobase_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,simulated_geobase_reference_file_0,685-77-0916,Betty,Audrey,Keel,19221205,47461,W ROXBURY DR,None,None,ANYTOWN,WA,00000
1,simulated_geobase_reference_file_1,765-44-4521,Ethep,Nancy,Collier,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,simulated_geobase_reference_file_2,765-44-4521,Ethep,Nancy,Collier,19230213,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,simulated_geobase_reference_file_3,726-57-2168,Josephine,Margaret,Babbie,19230728,3743,MESA VERDE ST,None,None,ANYTOWN,WA,00000
4,simulated_geobase_reference_file_4,726-57-2168,Josephine,Margaret,Babbie,19230728,3743,MESA VERDE ST,None,None,None,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32820,simulated_geobase_reference_file_32820,939-53-1702,Josiah,L,Mcmiller,NaN,2904,YACHT CLUB PT,None,None,ANYTOWN,WA,00000
32821,simulated_geobase_reference_file_32821,939-53-1702,Josiah,L,Mcmiller,NaN,W14066,E VALENCIA RD,None,None,ANYTOWN,WA,00004
32822,simulated_geobase_reference_file_32822,939-53-1702,Josiah,L,Mcmiller,NaN,W14066,E VALENCIA RD,None,None,ANYTOWN,WA,00000
32823,simulated_geobase_reference_file_32823,993-56-2286,Larisa,A,Beilfuss,NaN,1411,W 80TH ST,None,None,ANYTOWN,WA,00000


In [51]:
# Actual number of rows
len(simulated_geobase_reference_file)

32825

## Track ground truth for reference files

In [52]:
def get_simulants_of_source_records(source_record_pairs, filter_record_ids=None):
    if filter_record_ids is not None:
        source_record_pairs = source_record_pairs.pipe(filter_record_ids)

    result = (
        df_ops.drop_duplicates(source_record_pairs)
            .merge(source_record_ground_truth, on='source_record_id')
            [['record_id', 'simulant_id']]
            .pipe(df_ops.drop_duplicates)
    )
    result = result.merge(
        df_ops.groupby_agg_small_groups(result, by='record_id', agg_func=lambda x: x.simulant_id.nunique().rename('n_unique_simulants')).reset_index(),
        on='record_id',
        how='left',
    )
    return result

def get_ground_truth_for_records(source_record_pairs):
    result = df_ops.persist(get_simulants_of_source_records(source_record_pairs))

    result_nunique_describe = df_ops.compute(result['n_unique_simulants'].describe())

    if numpy.isclose(result_nunique_describe.loc['max'], 1):
        print('No collisions')
        return result

    print('Collisions:')
    display(result_nunique_describe)

    print('Simulated tax records with the most collisions:')
    most_collisions_record_id = result.sort_values('n_unique_simulants', ascending=False).head(1).record_id.iloc[0]
    most_collisions_source_record_ids = df_ops.persist(
        df_ops.drop_duplicates(source_record_pairs[source_record_pairs.record_id == most_collisions_record_id][['source_record_id']])
    )
    most_collisions_tax_filings = (
        most_collisions_source_record_ids
            .merge(df_ops.drop_duplicates(simulated_taxes_source_record_pairs[['record_id', 'source_record_id']]), on='source_record_id', how='inner')
            .merge(simulated_taxes, on='record_id', how='left')
            .merge(source_record_ground_truth, on='source_record_id', how='left')
    )
    display(df_ops.compute(most_collisions_tax_filings))

    print('Simulated SSA records with the most collisions:')
    display(
        df_ops.compute(
            most_collisions_source_record_ids
                .merge(simulated_ssa_numident.rename(columns={'record_id': 'source_record_id'}), on='source_record_id', how='inner')
                .merge(source_record_ground_truth, on='source_record_id', how='left')
        )
    )

    return result

### Simulated Census Numident

In [53]:
len(simulated_census_numident_source_record_pairs)

38271

In [54]:
simulated_census_numident_ground_truth = get_ground_truth_for_records(simulated_census_numident_source_record_pairs)

No collisions


In [55]:
simulated_census_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_census_numident_0,0_18309,1
1,simulated_census_numident_1,0_39,1
2,simulated_census_numident_2,0_18040,1
3,simulated_census_numident_3,0_6194,1
4,simulated_census_numident_4,0_16689,1
...,...,...,...
18464,simulated_census_numident_18437,0_2471,1
18465,simulated_census_numident_18439,0_8560,1
18466,simulated_census_numident_18456,0_9244,1
18467,simulated_census_numident_18458,0_4231,1


### Simulated Alternate Name Numident

In [56]:
len(simulated_alternate_name_numident_source_record_pairs)

19871

In [57]:
simulated_alternate_name_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_name_numident_source_record_pairs)

No collisions


In [58]:
simulated_alternate_name_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_alternate_name_numident_11,0_2471,1
1,simulated_alternate_name_numident_14,0_1366,1
2,simulated_alternate_name_numident_20,0_4621,1
3,simulated_alternate_name_numident_21,0_17500,1
4,simulated_alternate_name_numident_22,0_19772,1
...,...,...,...
18918,simulated_alternate_name_numident_19866,0_23602,1
18919,simulated_alternate_name_numident_19867,0_23588,1
18920,simulated_alternate_name_numident_19868,0_23589,1
18921,simulated_alternate_name_numident_19869,0_23596,1


### Alternate DOB Numident

In [59]:
len(simulated_alternate_dob_numident_source_record_pairs)

19871

In [60]:
simulated_alternate_dob_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_dob_numident_source_record_pairs)

No collisions


In [61]:
simulated_alternate_dob_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_alternate_dob_numident_2,0_19983,1
1,simulated_alternate_dob_numident_14,0_1366,1
2,simulated_alternate_dob_numident_26,0_10112,1
3,simulated_alternate_dob_numident_30,0_19743,1
4,simulated_alternate_dob_numident_34,0_12990,1
...,...,...,...
18724,simulated_alternate_dob_numident_19866,0_23602,1
18725,simulated_alternate_dob_numident_19867,0_23588,1
18726,simulated_alternate_dob_numident_19868,0_23589,1
18727,simulated_alternate_dob_numident_19869,0_23596,1


### Simulated Name/DOB Reference File

In [62]:
len(simulated_name_dob_reference_file_source_record_pairs)

42541

In [63]:
simulated_name_dob_reference_file_ground_truth = get_ground_truth_for_records(simulated_name_dob_reference_file_source_record_pairs)

No collisions


In [64]:
simulated_name_dob_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_name_dob_reference_file_15854,0_19979,1
1,simulated_name_dob_reference_file_17511,0_6846,1
2,simulated_name_dob_reference_file_15285,0_19983,1
3,simulated_name_dob_reference_file_15286,0_19983,1
4,simulated_name_dob_reference_file_18789,0_262,1
...,...,...,...
19870,simulated_name_dob_reference_file_19858,0_23319,1
19871,simulated_name_dob_reference_file_19862,0_1328,1
19872,simulated_name_dob_reference_file_19864,0_23720,1
19873,simulated_name_dob_reference_file_19868,0_5904,1


### Simulated GeoBase Reference File

In [65]:
len(simulated_geobase_reference_file_source_record_pairs)

160076

In [66]:
simulated_geobase_reference_file_ground_truth = get_ground_truth_for_records(simulated_geobase_reference_file_source_record_pairs)

Collisions:


count    34175.000000
mean         1.081346
std          0.285926
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          3.000000
Name: n_unique_simulants, dtype: float64

Simulated tax records with the most collisions:


,source_record_id,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,simulant_id
0,simulated_w2_1099_4693,simulated_taxes_43812,847-73-7350,Matthew,M,Varg,47912,queens ct,None,None,Anytown,WA,00000,0_22201
1,simulated_w2_1099_4694,simulated_taxes_43813,847-73-7350,Bailey,K,Varg,47912,queens ct,None,None,Anytown,WA,00000,0_22202
2,simulated_w2_1099_4695,simulated_taxes_43814,847-73-7350,John,M,Vargas,47912,queens ct,None,None,Anytown,WA,00000,0_22205


Simulated SSA records with the most collisions:


,source_record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,simulant_id
0,simulated_ssa_numident_18221,847-73-7350,John,Michael,Vargas,19760229,Male,creation,20250805,0_22205


In [67]:
simulated_geobase_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_geobase_reference_file_26168,0_730,1
1,simulated_geobase_reference_file_1,0_1366,1
2,simulated_geobase_reference_file_2,0_1366,1
3,simulated_geobase_reference_file_26970,0_1366,1
4,simulated_geobase_reference_file_26971,0_1366,1
...,...,...,...
34170,simulated_geobase_reference_file_9409,0_15423,1
34171,simulated_geobase_reference_file_21697,0_703,1
34172,simulated_geobase_reference_file_22042,0_16072,1
34173,simulated_geobase_reference_file_31296,0_23240,2


In [68]:
%xdel simulated_taxes
%xdel simulated_taxes_source_record_pairs

### Get ground truth by SSN

In [69]:
simulated_ssn_simulant_pairs = df_ops.persist(
    df_ops.concat([
        simulated_census_numident[["record_id", "ssn"]].merge(simulated_census_numident_ground_truth, on="record_id"),
        simulated_alternate_name_numident[["record_id", "ssn"]].merge(simulated_alternate_name_numident_ground_truth, on="record_id"),
        simulated_alternate_dob_numident[["record_id", "ssn"]].merge(simulated_alternate_dob_numident_ground_truth, on="record_id"),
        simulated_name_dob_reference_file[["record_id", "ssn"]].merge(simulated_name_dob_reference_file_ground_truth, on="record_id"),
        simulated_geobase_reference_file[["record_id", "ssn"]].merge(simulated_geobase_reference_file_ground_truth, on="record_id"),
    ])
        [['ssn', 'simulant_id']]
        .pipe(df_ops.drop_duplicates)
)
simulated_ssn_simulant_pairs

,ssn,simulant_id
16,027-46-6956,0_12990
45,182-87-9730,0_5212
51,614-82-7024,0_12734
53,331-74-9303,0_7558
55,044-86-3127,0_3858
...,...,...
34168,977-84-6791,0_22674
34169,997-95-0405,0_21507
34172,939-53-1702,0_7328
34173,993-56-2286,0_22299


In [70]:
# How many simulated SSNs have different simulant IDs
# contributing to them?
df_ops.compute((df_ops.groupby_agg_small_groups(simulated_ssn_simulant_pairs, by='ssn', agg_func=lambda x: x.simulant_id.nunique()) > 1).sum())

1084

## Save results

In [71]:
files = {
    'simulated_census_numident': (simulated_census_numident, simulated_census_numident_ground_truth),
    'simulated_alternate_name_numident': (simulated_alternate_name_numident, simulated_alternate_name_numident_ground_truth),
    'simulated_alternate_dob_numident': (simulated_alternate_dob_numident, simulated_alternate_dob_numident_ground_truth),
    'simulated_geobase_reference_file': (simulated_geobase_reference_file, simulated_geobase_reference_file_ground_truth),
    'simulated_name_dob_reference_file': (simulated_name_dob_reference_file, simulated_name_dob_reference_file_ground_truth),
}

In [72]:
simulated_reference_files = [
    simulated_census_numident,
    simulated_alternate_name_numident,
    simulated_alternate_dob_numident,
    simulated_geobase_reference_file,
    simulated_name_dob_reference_file,
]
# TODO: Rename the ssn column to explicitly include itins, since this is confusing
simulated_all_ssns_itins_in_reference_files = df_ops.concat([df[["ssn"]] for df in simulated_reference_files], ignore_index=True)
simulated_ssn_to_pik = (
    simulated_all_ssns_itins_in_reference_files
        .pipe(df_ops.drop_duplicates)
        .pipe(add_unique_id_col, col_name='pik')
        [['ssn', 'pik']]
)
simulated_ssn_to_pik

,ssn,pik
0,027-46-6956,88956
1,182-87-9730,88975
2,614-82-7024,88979
3,331-74-9303,88980
4,044-86-3127,88982
...,...,...
18974,977-84-6791,108816
18975,997-95-0405,108817
18976,939-53-1702,108818
18977,993-56-2286,108819


In [73]:
simulated_pik_simulant_pairs = (
    simulated_ssn_simulant_pairs
        .merge(simulated_ssn_to_pik, on="ssn", how="inner")
        [['pik', 'simulant_id']]
)
simulated_pik_simulant_pairs

,pik,simulant_id
0,88956,0_12990
1,88975,0_5212
2,88979,0_12734
3,88980,0_7558
4,88982,0_3858
...,...,...
20108,108816,0_22674
20109,108817,0_21507
20110,108818,0_7328
20111,108819,0_22299


In [74]:
%xdel simulated_ssn_simulant_pairs

In [75]:
for file_name, (file, ground_truth) in files.items():
    file = df_ops.persist(file.merge(simulated_ssn_to_pik, on='ssn', how='left'))
    assert df_ops.compute(file.pik.notnull().all())

    save_file_with_ground_truth(file_name, file, ground_truth)

In [76]:
simulated_pik_simulant_pairs_path = f'{output_dir}/{data_to_use}/simulated_pik_simulant_pairs.parquet'
utils.remove_path(simulated_pik_simulant_pairs_path)
simulated_pik_simulant_pairs.to_parquet(simulated_pik_simulant_pairs_path)